Authors:<br>
Joshua Cao<br>
Anthony Luu<br>
Winfield Zhang<br>

This is the final project for CS178 intro to machine learning. We were given 100,000 inputs and outputs, along with 100,000 test inputs without the outputs. The goal was to submit out predictions on the test input and achieve the best possible accuracy. 
<br><br>
We primarily used sklearn, although we did experiment with tensorflow and keras. We also used our professor's package, mltools.
<br><br>
We tested a lot of different classifiers, and averaged the predictions of the best performing ones. We were able to reach 76% accuracy. 

<font size="10">Imports</font>

In [1]:
import mltools as ml
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sklearn
import sklearn.metrics
import sklearn.ensemble
import sklearn.svm
import sklearn.naive_bayes
import sklearn.neural_network
import tensorflow as tf
import keras
import xgboost

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report


Using TensorFlow backend.


<font size="7">Helper functions</font>

In [77]:
def score_learner(lr):
    print("training")
    print(lr.score(Xtr, Ytr))
    p_tr = lr.predict(Xtr)
    print(sklearn.metrics.roc_auc_score(Ytr, lr.predict_proba(Xtr)[:,1]))
    print(classification_report(Ytr, p_tr))

    print("validation")
    print(lr.score(Xva, Yva))
    p_va = lr.predict(Xva)
    print(sklearn.metrics.roc_auc_score(Yva, lr.predict_proba(Xva)[:,1]))
    print(classification_report(Yva, p_va))
    

In [3]:
def write_to_file(filename, lr):
#     p_tr = lr.predict_proba(Xtr)
#     f = open("predictions_five/" + filename + "_tr.txt", "w")
#     for i in range(len(p_tr)):
#         f.write(str(p_tr[i][1]) + "\n")
#     f.close()
    
    p_va = lr.predict_proba(Xva)
    f = open("predictions_five/" + filename + "_va.txt", "w")
    for i in range(len(p_va)):
        f.write(str(p_va[i][1]) + "\n")
    f.close()
    
    p_te = lr.predict_proba(Xtest)
    f = open("predictions_five/" + filename + "_te.txt", "w")
    for i in range(len(p_te)):
        f.write(str(p_te[i][1]) + "\n")
    f.close()

<font size="10">Data Preprocessing</font>

In [88]:
X = np.genfromtxt('data/X_train.txt',delimiter=None)
Y = np.genfromtxt('data/Y_train.txt',delimiter=None)

Xtest = np.genfromtxt("data/X_test.txt", delimiter=None)

np.random.seed(0)
X,Y = ml.shuffleData(X,Y)


[[2.5000e+02 2.3300e+02 2.4308e+02 2.3454e+02 2.5780e+03 2.3100e+02
  0.0000e+00 1.6742e+01 5.5478e+00 6.0934e-01 2.7805e+00 2.2254e+00
  3.6493e+00 0.0000e+00]
 [2.3850e+02 2.2900e+02 2.4302e+02 2.3219e+02 4.3980e+03 4.6700e+02
  0.0000e+00 3.2346e+00 6.1774e+00 1.7202e+00 3.1890e+00 2.2173e+00
  3.0515e+00 0.0000e+00]
 [2.5000e+02 2.2600e+02 2.4070e+02 2.3199e+02 8.5080e+03 2.4850e+03
  0.0000e+00 1.4621e+00 7.1734e+00 2.1631e+00 2.3984e+00 1.2261e+00
  1.7826e+00 0.0000e+00]
 [2.4500e+02 2.4100e+02 2.4818e+02 2.4818e+02 6.1100e+02 0.0000e+00
  0.0000e+00 5.4439e+00 3.4458e+00 0.0000e+00 3.3276e+00 1.9390e+00
  2.0000e+01 0.0000e+00]
 [2.5100e+02 2.2250e+02 2.3766e+02 2.2936e+02 1.6150e+03 6.6500e+02
  0.0000e+00 1.6030e+00 8.3773e+00 3.4329e+00 5.2864e+00 2.4057e+00
  2.8455e+00 0.0000e+00]
 [2.2650e+02 2.0650e+02 2.2734e+02 2.1308e+02 1.9090e+03 1.3950e+03
  6.3200e+02 1.2247e+00 1.2494e+01 8.5875e+00 4.3209e+00 5.1717e+00
  1.9251e+00 1.4000e+00]
 [2.4200e+02 2.2000e+02 2.3727e+02

<font size="3">normalize</font>

In [66]:
Xmax = np.max(X, axis=0)
Xmin = np.min(X, axis=0)
X = (X - Xmin) / (Xmax - Xmin)

Xmax = np.max(Xtest, axis=0)
Xmin = np.min(Xtest, axis=0)
Xtest = (Xtest - Xmin) / (Xmax - Xmin)

<font size="3">split data</font>

In [67]:
Xtr = X[0:50000]
Ytr = Y[0:50000]
Xva = X[50000:100000]
Yva = Y[50000:100000]

print(Ytr.shape)

(50000,)


<font size="3">Under sample class 0</font>

<font size="2">This works well only for a few of the classifiers</font>

In [56]:
mask = Ytr == 0
Xmask0 = Xtr[mask,:]
Xmask1 = Xtr[~mask,:]
    
len_mask1 = len(Xmask1)
ones = np.asarray([1 for _ in range(len_mask1)])
zeros = np.asarray([0 for _ in range(len_mask1)])
Xtr = np.vstack((Xmask0[0:len_mask1], Xmask1))
Ytr = np.concatenate((zeros, ones))
                   
Xtr,Ytr = ml.shuffleData(Xtr,Ytr)
    
print(Xtr.shape)
print(Ytr.shape)

(34074, 14)
(34074,)


<font size="3">Select features</font>

In [522]:

# {A:0, B:0, C:2, D:3, E:4, F:5, G:6, H:7, I:8, J:9, K:10, L:11, M:12, N:13}

del_features = [4,6,7,10]
# del_features = [1,2,3,4,5,6,7,8,9,10,11]
# del_features = [1,3,4,5,6,7,8,9,10,11]

Xtr = np.delete(Xtr, del_features, axis = 1)
Xva = np.delete(Xva, del_features, axis = 1)

Xtest = np.delete(Xtest, del_features, axis = 1)

print(Xtr.shape)
print(Xtest.shape)

(47592, 10)
(100000, 10)


<font size="3">Output Yva to a file</font>

In [7]:
f = open("predictions_five/Y_validation.csv", "w")
for y in Yva:
    f.write(str(y) + "\n")
f.close()

<font size="10">AdaBoost Decision Trees</font>

<font size="5">Training</font>

In [70]:
ada_dt = sklearn.ensemble.AdaBoostClassifier(sklearn.tree.DecisionTreeClassifier(max_depth=3, max_features=7),
                         algorithm="SAMME.R",
                         n_estimators=1000,
                         learning_rate=.1)

ada_dt.fit(Xtr, Ytr)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=0.1, n_estimators=1000, random_state=None)

<font size="5">Scoring</font>

In [78]:
score_learner(ada_dt)

training
0.75106
0.8064658267064289
              precision    recall  f1-score   support

         0.0       0.75      0.93      0.83     32963
         1.0       0.75      0.41      0.53     17037

   micro avg       0.75      0.75      0.75     50000
   macro avg       0.75      0.67      0.68     50000
weighted avg       0.75      0.75      0.73     50000

validation
0.71032
0.7141545029274552
              precision    recall  f1-score   support

         0.0       0.73      0.90      0.80     32915
         1.0       0.64      0.35      0.45     17085

   micro avg       0.71      0.71      0.71     50000
   macro avg       0.68      0.62      0.63     50000
weighted avg       0.70      0.71      0.68     50000



<font size="5">Output</font>

In [14]:
write_to_file("adaboost_dt", ada_dt)

<font size="10">Adaboost Boost Naive Bayes</font>

<font size="5">Training</font>

In [68]:
nb = sklearn.ensemble.AdaBoostClassifier(sklearn.naive_bayes.GaussianNB(),
                         algorithm="SAMME.R",
                         n_estimators=100,
                         learning_rate=.1)

nb.fit(Xtr, Ytr)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=GaussianNB(priors=None, var_smoothing=1e-09),
          learning_rate=0.1, n_estimators=100, random_state=None)

<font size="5">Testing</font>

In [79]:
score_learner(nb)

training
0.68642
0.6407541759719991
              precision    recall  f1-score   support

         0.0       0.70      0.92      0.80     32963
         1.0       0.61      0.23      0.33     17037

   micro avg       0.69      0.69      0.69     50000
   macro avg       0.65      0.58      0.56     50000
weighted avg       0.67      0.69      0.64     50000

validation
0.68376
0.6387787843671617
              precision    recall  f1-score   support

         0.0       0.70      0.92      0.79     32915
         1.0       0.60      0.22      0.32     17085

   micro avg       0.68      0.68      0.68     50000
   macro avg       0.65      0.57      0.56     50000
weighted avg       0.66      0.68      0.63     50000



<font size="5">Output</font>

In [17]:
write_to_file("adaboost_nb", nb)

<font size="10">Bagged Trees</font>

<font size="5">Training</font>

In [18]:
class BaggedTree(ml.base.classifier):
    def __init__(self, learners):
        """Constructs a BaggedTree class with a set of learners. """
        self.learners = learners
    
    def predictSoft(self, X):
        """Predicts the probabilities with each bagged learner and average over the results. """
        n_bags = len(self.learners)
        preds = [self.learners[l].predictSoft(X) for l in range(n_bags)]
        return np.mean(preds, axis=0)
    
    def predict_proba(self, X):
        n_bags = len(self.learners)
        preds = [self.learners[l].predictSoft(X) for l in range(n_bags)]
        return np.mean(preds, axis=0)


In [19]:
n_bags = 100
bags = []
for i in range(n_bags):
#   print("bag: " + str(i))
  Xi, Yi = ml.bootstrapData(Xtr, Ytr, Xtr.shape[0])
  tree = ml.dtree.treeClassify(Xi,Yi,minParent=2**6,maxDepth=15,nFeatures=12)
  bags.append(tree)
    
bt = BaggedTree(bags)
bt.classes = np.unique(Y)

<font size="5">Test</font>

In [20]:
print("{0:>15}: {1:.4f}".format('Train AUC', bt.auc(Xtr, Ytr)))
print("{0:>15}: {1:.4f}".format('Validation AUC', bt.auc(Xva, Yva)))

      Train AUC: 0.8642
 Validation AUC: 0.7215


<font size="5">Output</font>

In [101]:
Ytr_hat = bt.predictSoft(Xtr)

# Xte = np.genfromtxt('data/X_test.txt',delimiter=None)
Ytr_hat = bt.predictSoft(Xtr)
f1 = open('predictions_five/baggedtree_tr.txt', 'w')
for i in range(len(Ytr_hat)):
  temp = Ytr_hat[i]
  f1.write(str(temp[1]) + '\n')
f1.close()

Ytr_hat = bt.predictSoft(Xva)
f1 = open('predictions_five/baggedtree_va.txt', 'w')
for i in range(len(Ytr_hat)):
  temp = Ytr_hat[i]
  f1.write(str(temp[1]) + '\n')
f1.close()

Ytr_hat = bt.predictSoft(Xtest)
f1 = open('predictions_five/baggedtree_te.txt', 'w')
for i in range(len(Ytr_hat)):
  temp = Ytr_hat[i]
  f1.write(str(temp[1]) + '\n')
f1.close()



<font size="10">Gradient Boost Decision Trees</font>

<font size="5">Training</font>

In [23]:
gb_dt = sklearn.ensemble.GradientBoostingClassifier(
                         n_estimators=50,
                         learning_rate=.1,
                         max_depth=10,
                         verbose=1)

gb_dt.fit(Xtr, Ytr)

      Iter       Train Loss   Remaining Time 
         1           1.2485           33.69s
         2           1.2179           35.52s
         3           1.1915           34.51s
         4           1.1680           34.27s
         5           1.1464           33.98s
         6           1.1276           33.65s
         7           1.1111           32.73s
         8           1.0960           32.02s
         9           1.0835           31.69s
        10           1.0713           30.74s
        20           0.9759           23.06s
        30           0.9193           14.44s
        40           0.8782            6.65s
        50           0.8430            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=50,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=1, warm_start=False)

<font size="5">Test</font>

In [80]:
score_learner(gb_dt)

training
0.82796
0.9148625175337015
              precision    recall  f1-score   support

         0.0       0.81      0.97      0.88     32963
         1.0       0.91      0.55      0.69     17037

   micro avg       0.83      0.83      0.83     50000
   macro avg       0.86      0.76      0.78     50000
weighted avg       0.84      0.83      0.82     50000

validation
0.71574
0.7281620278303063
              precision    recall  f1-score   support

         0.0       0.73      0.90      0.81     32915
         1.0       0.65      0.36      0.46     17085

   micro avg       0.72      0.72      0.72     50000
   macro avg       0.69      0.63      0.63     50000
weighted avg       0.70      0.72      0.69     50000



<font size="5">Output</font>

In [25]:
write_to_file("gradientboost_dt", gb_dt)

<font size="10">Neural Network</font>

<font size="5">training</font>

In [26]:
nnet = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(200,150,100), activation="relu", solver="adam", 
                                            learning_rate_init=0.001, max_iter=50, warm_start=True)
nnet.fit(Xtr, Ytr)

C:\Users\caojo\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 150, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=50, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=True)

<font size="5">testing</font>

In [81]:
score_learner(nnet)

training
0.70654
0.7007307703821006
              precision    recall  f1-score   support

         0.0       0.71      0.93      0.81     32963
         1.0       0.67      0.28      0.39     17037

   micro avg       0.71      0.71      0.71     50000
   macro avg       0.69      0.60      0.60     50000
weighted avg       0.70      0.71      0.67     50000

validation
0.70426
0.6956112602093945
              precision    recall  f1-score   support

         0.0       0.71      0.93      0.80     32915
         1.0       0.66      0.27      0.39     17085

   micro avg       0.70      0.70      0.70     50000
   macro avg       0.69      0.60      0.60     50000
weighted avg       0.69      0.70      0.66     50000



<font size="5">output</font>

In [28]:
write_to_file("nnet", nnet)

<font size="10">KNN</font>

<font size="5">Training</font>

In [29]:
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 50, weights="distance")
knn.fit(Xtr, Ytr) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=50, p=2,
           weights='distance')

<font size="5">Test</font>

In [30]:
score_learner(knn)

training
0.97658
0.9675478355692156
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     32963
         1.0       0.99      0.94      0.96     17037

   micro avg       0.98      0.98      0.98     50000
   macro avg       0.98      0.97      0.97     50000
weighted avg       0.98      0.98      0.98     50000

validation
0.70204
0.6099154929928103
              precision    recall  f1-score   support

         0.0       0.72      0.90      0.80     32915
         1.0       0.63      0.32      0.42     17085

   micro avg       0.70      0.70      0.70     50000
   macro avg       0.67      0.61      0.61     50000
weighted avg       0.69      0.70      0.67     50000



<font size="5">Output</font>

In [31]:
write_to_file("knn", knn)

<font size="10">Random Forest</font>

<font size="5">Training</font>

In [86]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, min_samples_leaf=5, oob_score=True, verbose=1)
rf.fit(Xtr, Ytr)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  1.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=True, random_state=None, verbose=1, warm_start=False)

<font size="5">Testing</font>

In [87]:
score_learner(rf)

training


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.86516


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.9598732951440566
              precision    recall  f1-score   support

         0.0       0.85      0.97      0.90     32963
         1.0       0.91      0.67      0.77     17037

   micro avg       0.87      0.87      0.87     50000
   macro avg       0.88      0.82      0.84     50000
weighted avg       0.87      0.87      0.86     50000

validation


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   11.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.72314


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7398755727665788
              precision    recall  f1-score   support

         0.0       0.74      0.90      0.81     32915
         1.0       0.67      0.38      0.48     17085

   micro avg       0.72      0.72      0.72     50000
   macro avg       0.70      0.64      0.65     50000
weighted avg       0.71      0.72      0.70     50000



[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.9s finished


<font size="5">Output</font>

In [34]:
write_to_file("randomforest", rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    6.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    6.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    9.7s finished


<font size="10">XGBoost decision trees</font>

<font size="5">Training</font>

In [35]:
xgb = xgboost.XGBClassifier(n_estimators=200, max_depth=15)
xgb.fit(Xtr,Ytr)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=15, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

<font size="5">Testing</font>

In [83]:
score_learner(xgb)

training
0.94422
0.9912708292670929
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     32963
         1.0       0.96      0.87      0.91     17037

   micro avg       0.94      0.94      0.94     50000
   macro avg       0.95      0.93      0.94     50000
weighted avg       0.94      0.94      0.94     50000

validation
0.71668
0.7368386250072296
              precision    recall  f1-score   support

         0.0       0.75      0.86      0.80     32915
         1.0       0.62      0.45      0.52     17085

   micro avg       0.72      0.72      0.72     50000
   macro avg       0.68      0.65      0.66     50000
weighted avg       0.70      0.72      0.70     50000



<font size="5">Output</font>

In [37]:
write_to_file("xgboost_dt", xgb)

<font size="10">Ensemble Predictions</font>

<font size="3">Testing average predictions and majority vote with different weights for each class</font>

Best: .7314

classifiers = [
    [ada_dt, 0],
    [nb, 1],
    [bt, 0],
    [gb_dt, 6],
    [nnet, 0],
    [knn, 0],
    [rf, 5],
    [xgb, 5],
]

In [95]:
classifiers = [
    [ada_dt, 0],
    [nb, 1],
    [bt, 0],
    [gb_dt, 6],
    [nnet, 0],
    [knn, 0],
    [rf, 5],
    [xgb, 5],
]

<font size="5">Test ensemble accuracy on validation data</font>

In [39]:
predicts = []

for classifier in classifiers:
    print("testing: " + str(type(classifier[0])))
    if classifier[0] == bt:
        predict = BaggedTree.predict_proba(classifier[0], Xva)
    else:
        predict = classifier[0].predict_proba(Xva)
    predict_binary = []
    for p in predict:
        if p[0] >= .5:
            predict_binary.append(0)
        else:
            predict_binary.append(1)
    print(sklearn.metrics.accuracy_score(Yva, predict_binary))
    predicts.append(predict)

testing: <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>
0.703
testing: <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>
0.68376
testing: <class '__main__.BaggedTree'>
0.7135
testing: <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
0.71574
testing: <class 'sklearn.neighbors.classification.KNeighborsClassifier'>
0.70204
testing: <class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>
0.70426
testing: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    5.4s finished


0.723
testing: <class 'xgboost.sklearn.XGBClassifier'>
0.71668


In [96]:
result = [[] for _ in range(len(Xva))]
for i in range(len(Xva)):
    for j in range(len(classifiers)):
        for _ in range(classifiers[j][1]):
            result[i].append(predicts[j][i][1])


In [97]:
print(np.array(result).shape)

means = np.mean(result, axis=1)
print(means.shape)

means_categorical = []
for i in range(len(means)):
    means_categorical.append(round(means[i]))

# print(sklearn.metrics.accuracy_score(Yva, bt.predict(Xva)))
print(sklearn.metrics.accuracy_score(Yva, means_categorical))
print(sklearn.metrics.roc_auc_score(Yva, means))
print(classification_report(Yva, means_categorical))



(50000, 7)
(50000,)
0.72022
0.7387395456526377
              precision    recall  f1-score   support

         0.0       0.74      0.89      0.81     32915
         1.0       0.65      0.39      0.48     17085

   micro avg       0.72      0.72      0.72     50000
   macro avg       0.70      0.64      0.65     50000
weighted avg       0.71      0.72      0.70     50000



<font size="5">Get predictions on test data</font>

In [42]:
test_predicts = []

for classifier in classifiers:
    print("testing: " + str(type(classifier[0])))
    if classifier[0] == bt:
        predict = BaggedTree.predict_proba(classifier[0], Xtest)
    else:
        predict = classifier[0].predict_proba(Xtest)
    test_predicts.append(predict)

testing: <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>
testing: <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>
testing: <class '__main__.BaggedTree'>
testing: <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
testing: <class 'sklearn.neighbors.classification.KNeighborsClassifier'>
testing: <class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>
testing: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    8.4s finished


testing: <class 'xgboost.sklearn.XGBClassifier'>


In [98]:
print(np.array(test_predicts).shape)
result = [[] for _ in range(len(Xtest))]
for i in range(len(Xtest)):
    for j in range(len(classifiers)):
        for _ in range(classifiers[j][1]):
            result[i].append(test_predicts[j][i][1])

(8, 100000, 2)


In [99]:
print(np.array(result).shape)

means = np.mean(result, axis=1)
print(means.shape)


(100000, 7)
(100000,)


Save the predictions to a file

In [100]:
# f = open("predictions_four/kaggle_submit_2.txt", "w")
# f.write("ID,prob1\n")
# for i in range(len(means)):
#     f.write(str(i) + ", " + str(round(means[i],2)) + "\n")
# f.close()

Yte = np.vstack((np.arange(means.shape[0]), means)).T
print(Yte.shape)
np.savetxt('predictions_five/Y_submit.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')


(100000, 2)


<font size="5">Testing majority predictions on validaiton data</font>

In [46]:
classifiers = [ada_dt, bt, gb_dt, knn, nnet, rf, xgb]
majority_predict = []
for i in range(len(predicts[0])):
    num_zero = 0
    for j in range(len(predicts)):
        if predicts[j][i][1] <= 0.5:
            num_zero += 1
    if num_zero > len(classifiers) / 2:
        majority_predict.append(0)
    else:
        majority_predict.append(1)
        
print(sklearn.metrics.accuracy_score(Yva, majority_predict))
print(sklearn.metrics.roc_auc_score(Yva, majority_predict))


0.71856
0.6194368828356897


In [47]:
print(classification_report(Yva, majority_predict))

              precision    recall  f1-score   support

         0.0       0.72      0.93      0.81     32915
         1.0       0.70      0.31      0.43     17085

   micro avg       0.72      0.72      0.72     50000
   macro avg       0.71      0.62      0.62     50000
weighted avg       0.71      0.72      0.68     50000



Write majority vote predictions to a file, did not end up using this

In [48]:
classifiers = [ada_dt, bt, gb_dt, knn, nnet, rf, xgb]
majority_predict = []
for i in range(len(test_predicts[0])):
    num_zero = 0
    for j in range(len(test_predicts)):
        if test_predicts[j][i][1] <= 0.5:
            num_zero += 1
    if num_zero > len(classifiers) / 2:
        majority_predict.append(0)
    else:
        majority_predict.append(1)
        

In [49]:
num_zero = 0
for p in majority_predict:
    if p == 0:
        num_zero += 1
print(num_zero)

81137


In [50]:
f = open("predictions_five/kaggle_submit_majority_vote.txt", "w")
f.write("ID,prob1\n")
for i in range(len(majority_predict)):
    f.write(str(i) + ", " + str(round(majority_predict[i],2)) + ".0\n")
f.close()